<a href="https://colab.research.google.com/github/rihemmaarefe/testwithoutdataset/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
# !pip install streamlit

In [161]:
# !npm install localtunnel

In [162]:
# !pip install river

In [163]:
# !pip install seaborn

In [164]:
# !pip install pysqlite3

In [165]:
# !pip install sql

In [166]:
# %%writefile app.py

In [167]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import seaborn as sns
import altair as alt
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from datetime import datetime
from PIL import Image

# ML pkgs
from river.naive_bayes import MultinomialNB
from river.feature_extraction import BagOfWords,TFIDF
from river.compose import Pipeline 


# Training data
data = [
    ("i hate you","subjective"),
    ("The pizza at that restaurant is delicious","subjective"),
    ("That painting is beautiful","subjective"),
    ("She is the best boss I've ever had","subjective"),
    ("Summer is the best season of the year","subjective"),
    ("That book is really interesting","subjective"),
    ("I feel happy when I'm with my friends","subjective"),
    ("we are cooler","subjective"),
    ("The sky is blue","objective"),
    ("The temperature outside is 72 degrees Fahrenheit","objective"),
    ("The Earth revolves around the Sun","objective"),
    ("Oxygen is essential for human survival","objective"),
    ("The capital of France is Paris","objective"),
    ("Water is a transparent, odorless, tasteless liquid that is essential for most forms of life on Earth. It is composed of two hydrogen atoms and one oxygen atom and has the chemical formula H2","objective"),
    ("The average lifespan of a domestic cat is 12 to 15 years","objective"),
    ("The Mona Lisa is a painting by Leonardo da Vinci","objective"),
    ("A chair is a piece of furniture designed for sitting on, typically consisting of a seat, backrest, and sometimes armrests, supported by four legs or a single pedestal","objective"),
    ("Jupiter is the largest planet in our solar system.","objective"),
    ("It refers to a specific type of toy, typically a small, often anatomically correct figurine or representation of a human or animal. The meaning of the word is not influenced by personal opinions, feelings, or biases, which makes it an objective term","objective")
]

#model
model = Pipeline(('vectorizer',BagOfWords(lowercase = True)),('nv',MultinomialNB()))
for x,y in data:
  model = model.learn_one(x,y)

#Storage
import sqlite3
conn = sqlite3.connect('data.db')
c = conn.cursor()

def create_table():
  c.execute('CREATE TABLE IF NOT EXISTS predictionTable(message TEXT,prediction TEXT,probability NUMBER,objective_proba NUMBER,subjective_proba NUMBER,postdate DATE)')


def add_data(message,prediction,probability,objective_proba,subjective_proba,postdate):
    c.execute('INSERT INTO predictionTable(message,prediction,probability,objective_proba,subjective_proba,postdate) VALUES (?,?,?,?,?,?)',(message,prediction,probability,objective_proba,subjective_proba,postdate))
    conn.commit()

def view_all_data():
	c.execute("SELECT * FROM predictionTable")
	data = c.fetchall()
	return data


menu = ["Home","Manage","About"]
create_table()

choice = st.sidebar.selectbox("Menu",menu)
if choice == "Home":
  st.subheader("Home")
  with st.form(key='mlform'):
    col1,col2 = st.columns([2,1])
    with col1:
      message = st.text_area("Message")
      submit_message = st.form_submit_button(label = 'Predict')
    with col2:
      st.write("********************")  
      st.write("Predict Text as Objective or Subjective") 
      st.write("********************")  

  if submit_message:
    prediction = model.predict_one(message)
    prediction_proba = model.predict_proba_one(message)
    probability = max(prediction_proba.values())
    postdate = datetime.now()
    
    add_data(message,prediction,probability,prediction_proba['objective'],prediction_proba['subjective'],postdate)
    st.success("Data Submitted")
    
    res_col1 ,res_col2 = st.columns(2)
    with res_col1:
      st.info("Original Text")
      st.write(message)
      
      st.success("Prediction")
      st.write(prediction)
    with res_col2:
      st.info("Probability")
      #  plot
      df_proba = pd.DataFrame({'label':prediction_proba.keys(),'probability':prediction_proba.values()})
      # visz
      fig = alt.Chart(df_proba).mark_bar().encode(x='label',y='probability')
      st.altair_chart(fig,use_container_width=True)	





elif choice == "Manage":
	st.subheader("Manage & Monitor Results")
	stored_data =  view_all_data() 
	new_df = pd.DataFrame(stored_data,columns=['message','prediction','probability','objective_proba','subjective_proba','postdate'])
	st.dataframe(new_df)
	new_df['postdate'] = pd.to_datetime(new_df['postdate'])

	# c = alt.Chart(new_df).mark_line().encode(x='minutes(postdate)',y='probability')# For Minutes
	c = alt.Chart(new_df).mark_line().encode(x='postdate',y='probability')
	st.altair_chart(c)
	
	c_objective_proba = alt.Chart(new_df['objective_proba'].reset_index()).mark_line().encode(x='objective_proba',y='index')
	c_subjective_proba = alt.Chart(new_df['subjective_proba'].reset_index()).mark_line().encode(x='subjective_proba',y='index')
		
		

	c1,c2 = st.columns(2)
	with c1:
		with st.expander("Objective Probability"):
			st.altair_chart(c_objective_proba,use_container_width=True)

	with c2:
		with st.expander("Subjective Probability"):
			st.altair_chart(c_subjective_proba,use_container_width=True)


	with st.expander("Prediction Distribution"):
		fig2 = plt.figure()
		sns.countplot(x='probability',data=new_df)
		st.pyplot(fig2)
		


else:
  st.subheader("About")

  col1,col2 = st.columns([2,1])
  with col1:
    st.write("********************")  
    st.write("Made by Rihem Maaref")
    st.write("Predict Text as Objective or Subjective")
    st.write("Using Machine Learning")
    st.write("With River and Naive Bayes")
    st.write("********************")  

  with col2:
    st.write("********************")  
    image = Image.open('/content/drive/MyDrive/Stage/Exercises.jpg')
    st.image(image)
    st.write("********************") 


Overwriting app.py


In [168]:
!ls

app.py	data.db  drive	logs.txt  node_modules	package-lock.json  sample_data


In [169]:
# !pip install pyngrok

In [170]:
!streamlit run /content/app.py &>/content/logs.txt &

In [171]:
!npx localtunnel --port 8501

npx: installed 22 in 3.954s
your url is: https://solid-islands-boil-35-230-171-20.loca.lt
^C
